# Criação de chat básico com RAG (Retrieval-Augmented Generation)

In [ ]:
! pip install -U  openai  langchain  langchain_openai  langchain-community  python-dotenv  datasets  qdrant-client  tiktoken

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv('.env')

True

## AzureChatOpenAI

No tutorial foi usado a API da OpenAI diretamente e aqui foi usada a AzureOpenAI.

### Criação do Chatbot 

Esse é um chat básico que consome o gpt 3.5 turbo 16k fornecido pela API da AzureChatOpenAI.

Instânciamento do chat.

In [64]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo-16k",  
    openai_api_version="2023-06-01-preview",  
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"), 
)

Do LangChain Schema são importados 3 elementos:
- SystemMessage: Usado para definir escopo e tom do modelo;
- HumanMessage: Define o conteúdo da mensagem humana;
- AIMessage: Resposta do modelo.

A lista messages é como um log do chat com o LLM. Todo o log é passado para o llm com o método invoke para questão de contexto.

In [65]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
## messages é a lista que corresponde ao Log da conversa
messages = [
    SystemMessage(content="Você é um assistente útil que responde perguntas."),
    HumanMessage(content="Olá Bot, como você está hoje?"),
    AIMessage(content="Estou bem, obrigado. Como posso ajudar?"),
    HumanMessage(content="Gostaria de entender o que é machine learning.")
]


response = llm.invoke(messages)
print(response.content)

Claro! Machine learning (aprendizado de máquina, em português) é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos de computador capazes de aprender e tomar decisões ou fazer previsões com base em dados sem serem explicitamente programados.

A ideia central do machine learning é permitir que os computadores aprendam com exemplos e experiências passadas para melhorar seu desempenho em tarefas futuras. Os modelos de machine learning são treinados usando conjuntos de dados, nos quais são apresentados exemplos e uma função objetivo que descreve o que se deseja aprender ou prever.

Existem diversas abordagens de machine learning, como aprendizado supervisionado, não supervisionado, por reforço, entre outras. No aprendizado supervisionado, o modelo aprende a partir de pares de entrada e saída rotulados, enquanto no aprendizado não supervisionado, o modelo busca por padrões e estrutura nos dados sem rótulos prévios.

O machine learning tem apli

Adicionando a resposta "response" ao log "messages". Isso é feito para manter o contexto da conversa.

In [41]:
messages.append(response)

Novo Prompt (HumanMessage) que é adicionado ao log (messages) através de append.

In [42]:
prompt = HumanMessage (
    content='Qual é a diferença entre aprendizado supervisionado e não supervisionado?'
)
messages.append(prompt)

In [44]:
response = llm.invoke(messages)
print(response.content)

No Aprendizado Supervisionado, os algoritmos são treinados usando pares de dados de entrada e a resposta esperada correspondente. Isso significa que o conjunto de dados de treinamento contém exemplos rotulados, onde cada entrada tem uma resposta conhecida associada. O objetivo do modelo de Machine Learning é aprender a mapear as entradas para as saídas corretas, de modo que possa fazer previsões ou classificar novos dados.

Por exemplo, em um problema de classificação de e-mails em spam e não spam, o modelo de Aprendizado Supervisionado é treinado com e-mails rotulados como spam ou não spam. Com base nesses exemplos, o modelo aprende os padrões que distinguem os dois tipos de e-mails e pode classificar corretamente novos e-mails como spam ou não spam.

Já no Aprendizado Não Supervisionado, não existem rótulos ou respostas esperadas nos conjuntos de dados de treinamento. Os algoritmos de Aprendizado Não Supervisionado exploram a estrutura interna dos dados para encontrar padrões, agrupa

## Hallucination

##### Teste 1, sem contexto do RAG:

In [45]:
messages.append(response)

prompt = HumanMessage(
    content='O que tem de tão especial no Mistral 7B?'
)

messages.append(prompt)

response = llm.invoke(messages)

In [47]:
print(response.content)

Desculpe, mas não estou familiarizado com uma referência específica ao "Mistral 7B". Pode ser um termo relacionado a algo específico como um produto, lugar ou uma designação que estou desconhecido. Poderia fornecer mais informações ou contexto para que eu possa entender melhor e ajudar?


#### Teste 2, também sem contexto:

In [48]:
messages.append(response)

prompt = HumanMessage(
    content='Você pode me falar sobre o LLMChain no LangChain?'
)

messages.append(prompt)

response = llm.invoke(messages)

In [49]:
print(response.content)

Peço desculpas, mas não tenho informações sobre o "LLMChain no LangChain". Não tenho conhecimento de uma referência específica a esses termos. Poderia fornecer mais detalhes ou contexto para que eu possa ajudar de forma mais precisa? Estou aqui para responder suas perguntas da melhor maneira possível.


## Context injection

Aqui o contexto é adicionado ao prompt manualmente. A fonte externa de dados são os dois parágrafos dentro da lista llmchain_information.

In [50]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

In [ ]:
source_knowledge = "\n".join(llmchain_information)


1020

### Prompt enriquecido manualmente

A query equivale ao input do usuário e o augmented_prompt é composto de uma espécie de System Message para o modelo, um contexto que é a fonte externa e a pergunta (query).

In [52]:
query = 'Você pode me falar sobre o LLMChain no LangChain?'

augmented_prompt = f"""Use o contexto abaixo para responder à pergunta

Contexto:
{source_knowledge}

Pergunta: {query}"""

Adicionando o augmented_prompt ao log de mensagens, passando o log ao modelo e exibindo o content da resposta.

In [53]:
prompt = HumanMessage(
    content=augmented_prompt
)

messages.append(prompt)

response = llm.invoke(messages)
print(response.content)

O LLMChain no LangChain é um componente central e comum na estrutura do LangChain. É um tipo de cadeia (chain) que consiste em um PromptTemplate, um modelo (seja um LLM ou um ChatModel) e um analisador de saída opcional. Essa cadeia (chain) recebe várias variáveis de entrada, usa o PromptTemplate para formatá-las em uma estimulação e, em seguida, passa essa estimulação para o modelo. Por fim, usa o analisador de saída (se fornecido) para converter a saída do LLM (Linguagem de Modelo de Aprendizado) em um formato final.

A estrutura das cadeias (chains) no LangChain é altamente modular e flexível, permitindo a combinação de componentes de forma específica para atender a diferentes casos de uso. O LangChain é um framework para desenvolver aplicações alimentadas por modelos de linguagem, buscando conectar o modelo de linguagem com várias fontes de dados e possibilitar a interação desse modelo com o ambiente em que opera.

A combinação de cadeias de LLM e outros componentes no LangChain pe

## RAG

### Embeddings 

dataset usado - Mistral 7b on Hugging Face:  https://huggingface.co/datasets/infoslack/mistral-7b-arxiv-paper-chunked

In [131]:
from langchain_openai import AzureOpenAIEmbeddings

In [ ]:
# O embedding não funciona caso não haja esse "tratamento" antes.
if "OPENAI_API_BASE" in os.environ:
    del os.environ["OPENAI_API_BASE"]

# Inicialize o AzureOpenAIEmbeddings
embed_model = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment='text-embedding-3-small-2',  # modelo de recuperação usado
    openai_api_version="2023-05-15",
    api_key=os.environ["AZURE_OPENAI_KEY"],
    chunk_size=1000,  # Adicione o parâmetro chunk_size
)

In [159]:
texts = [
    'this is one chunk',
    'this is the second chunk of text'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

### Preparando fonte de conhecimento externa

Carregamento dos dados

In [ ]:
from datasets import load_dataset
dataset = load_dataset('infoslack/mistral-7b-arxiv-paper-chunked', split="train")
dataset[0]

Conversão do conjunto para um dataframe do Pandas

In [ ]:
data = dataset.to_pandas()
data.head()

Para o exemplo só precisaremos de duas colunas, chunk e source, então isolamos essas

In [163]:
docs = data[['chunk', 'source']]
docs.head()

,chunk,source
0,"Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayr...",http://arxiv.org/pdf/2310.06825
1,automated benchmarks. Our models are released ...,http://arxiv.org/pdf/2310.06825
2,GQA significantly accelerates the inference sp...,http://arxiv.org/pdf/2310.06825
3,Mistral 7B takes a significant step in balanci...,http://arxiv.org/pdf/2310.06825
4,parameters of the architecture are summarized ...,http://arxiv.org/pdf/2310.06825


Usamos o método load da classe DataFrameLoader do pacote document_loaders para carregar os registos no formado de documento do LangChain

In [ ]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(docs, page_content_column='chunk')
documents = loader.load()

In [169]:
documents[0]

Document(metadata={'source': 'http://arxiv.org/pdf/2310.06825'}, page_content='Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,\nDevendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel,\nGuillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux,\nPierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix,\nWilliam El Sayed\nAbstract\nWe introduce Mistral 7B, a 7–billion-parameter language model engineered for\nsuperior performance and efficiency. Mistral 7B outperforms the best open 13B\nmodel (Llama 2) across all evaluated benchmarks, and the best released 34B\nmodel (Llama 1) in reasoning, mathematics, and code generation. Our model\nleverages grouped-query attention (GQA) for faster inference, coupled with sliding\nwindow attention (SWA) to effectively handle sequences of arbitrary length with a\nreduced inference cost. We also provide a model fine-tuned to follow instructions,\nMistral 7B – Inst

In [166]:
documents[0].page_content

'Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,\nDevendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel,\nGuillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux,\nPierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix,\nWilliam El Sayed\nAbstract\nWe introduce Mistral 7B, a 7–billion-parameter language model engineered for\nsuperior performance and efficiency. Mistral 7B outperforms the best open 13B\nmodel (Llama 2) across all evaluated benchmarks, and the best released 34B\nmodel (Llama 1) in reasoning, mathematics, and code generation. Our model\nleverages grouped-query attention (GQA) for faster inference, coupled with sliding\nwindow attention (SWA) to effectively handle sequences of arbitrary length with a\nreduced inference cost. We also provide a model fine-tuned to follow instructions,\nMistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and\nautomated ben

In [167]:
documents[0].metadata

{'source': 'http://arxiv.org/pdf/2310.06825'}

Todos esses dados trabalhados são adicionados ao banco de dados vetorizado

In [170]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(model='text-embedding-3-small-2')

qdrant = Qdrant.from_documents(
    documents=documents,
    embedding=embeddings,
    location=':memory:',
    collection_name="chatbot"
)

Consulta a ser feita da base

In [171]:
query= "O que tem de tão especial no Mistral 7B?"

Usa-se o método de busca de similaridade do qdrant para passar a query ao banco vetorizado e o parâmetro k define o limite de resposta que, no caso, são 3 chunks.<br>
O retorno serão os 3 chunks mais similares ao que foi perguntado.

In [ ]:
qdrant_search = qdrant.similarity_search(query, k=3)

# Estou lendo o page_content do primeiro chunk da lista. O método similarity_search retorna 3 chunks dentro de uma lista e para visualizar a mensagem de cada é preciso acessar o índice correspondente.
qdrant_search[0].page_content

'Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,\nDevendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel,\nGuillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux,\nPierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix,\nWilliam El Sayed\nAbstract\nWe introduce Mistral 7B, a 7–billion-parameter language model engineered for\nsuperior performance and efficiency. Mistral 7B outperforms the best open 13B\nmodel (Llama 2) across all evaluated benchmarks, and the best released 34B\nmodel (Llama 1) in reasoning, mathematics, and code generation. Our model\nleverages grouped-query attention (GQA) for faster inference, coupled with sliding\nwindow attention (SWA) to effectively handle sequences of arbitrary length with a\nreduced inference cost. We also provide a model fine-tuned to follow instructions,\nMistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and\nautomated ben

A função custom_prompt usa o resultado da busca por similaridade para fazer um prompt enriquecido onde é passado o comando, o contexto e a pergunta que estará na variável "query".

In [175]:
def custom_prompt (query: str):
    results = qdrant.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])
    augmented_prompt = f"""Use o contexto abaixo para responder à pergunta.

    Contexto: 
    {source_knowledge}

    Pergunta: {query}"""
    return augmented_prompt

Exemplo de como o prompt enriquecido ficará.

In [176]:
print(custom_prompt(query))

Use o contexto abaixo para responder à pergunta.

    Contexto: 
    Mistral 7B
Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,
Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel,
Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux,
Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix,
William El Sayed
Abstract
We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Lla

In [177]:
prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)

response = llm.invoke(messages)

In [178]:
print(response.content)

O Mistral 7B é um modelo de linguagem com 7 bilhões de parâmetros que foi projetado para ter um desempenho superior e eficiência. Ele supera os melhores modelos abertos de 13B (Llama 2) em todos os benchmarks avaliados e o melhor modelo lançado de 34B (Llama 1) em áreas como raciocínio, matemática e geração de código. O modelo utiliza atenção agrupada por consulta (GQA) para acelerar o processo de inferência e atenção de janela deslizante (SWA) para lidar de forma eficaz com sequências de comprimento arbitrário com um custo de inferência reduzido. Além disso, também é fornecido um modelo ajustado para seguir instruções, o Mistral 7B - Instruir, que supera o modelo de chat Llama 2 13B em benchmarks humanos e automatizados. O Mistral 7B foi projetado para ser adaptável e facilmente ajustável a várias tarefas. Sua eficiência e desempenho aprimorados são alcançados por meio desses mecanismos de atenção. Ele é licenciado sob a Apache 2.0 e possui implementação de referência para facilitar o

## Groq

Usando o modelo de "unidade de processamento de linguagem" (LPU) que gera respostas de IA extremamente rápido.<br>

Instalação pip

In [ ]:
! pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Importação da classe a partir do langchain_groq.<br>
Para esse chat o modelo usado foi o DeepSeek r1 de 70 bilhões de parâmetros.

In [ ]:
from langchain_groq import ChatGroq

# if "GROQ_API_KEY" not in os.environ:
#     os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")
groq_api_key = os.environ["GROQ_API_KEY"]

chat = ChatGroq(temperature=0, model_name="deepseek-r1-distill-llama-70b", )

In [186]:
prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)
response = chat.invoke(messages)
print(response.content)

<think>
Ok, o usuário está perguntando o que tem de tão especial no Mistral 7B. Primeiro, preciso lembrar que o contexto fornecido inclui detalhes técnicos sobre o modelo. O usuário pode estar interessado em entender por que o Mistral 7B se destaca em relação a outros modelos, como o Llama 2 ou Llama 1.

Vou começar analisando o contexto. Vejo que o Mistral 7B tem 7 bilhões de parâmetros e foi projetado para desempenho superior e eficiência. Isso já é um ponto importante, pois muitos modelos grandes sacrificam a eficiência pelo desempenho.

Em seguida, noto que ele supera o Llama 2 13B em todos os benchmarks avaliados e o Llama 1 34B em raciocínio, matemática e geração de código. Isso mostra que, apesar de ser menor em tamanho, o Mistral 7B é mais eficiente e performático em certas áreas.

Outro ponto-chave são as técnicas de atenção utilizadas: Grouped-Query Attention (GQA) e Sliding Window Attention (SWA). A GQA acelera a velocidade de inferência e reduz os requisitos de memória, per